# Hydrological modelling module

<div class="alert alert-info"> <b>INFO</b>
`xhydro` provides tools to execute and calibrate hydrological models, but will not prepare the model itself. This should be done beforehand.
</div>

`xhydro` provides a collection of functions that can serve as the main entry point for hydrological modelling. The entire framework is based on the `xh.modelling.hydrological_model` function and its `model_config` dictionary, which is meant to contain all necessary information to execute the given hydrological model. For example, depending on the model, it can store meteorological datasets directly, paths to datasets (netCDF files or other), csv configuration files, parameters, and basically anything that is required to configure and execute an hydrological model.

It then becomes the User's responsibility to ensure that all required information for a given model are provided in the `model_config` object, both in the data preparation stage and in the hydrological model implementation. This can be addressed by calling the `xh.modelling.get_hydrological_model_inputs` function to get a list of the required keys for a given model, as well as the documentation. Parameters for that function are:
- `model_name`: As listed below.
- `required_only`: Whether to return all possible inputs, or only the required ones.

Currently available models are:
- `Hydrotel`
- `Dummy` (only used for testing purposes)

In [ ]:
import xhydro as xh
import xhydro.modelling as xhm

# This function can be called to get a list of the keys for a given model, as well as its documentation.
inputs, docs = xhm.get_hydrological_model_inputs("Hydrotel", required_only=False)

inputs

In [ ]:
print(docs)

In [ ]:
# Workaround for determining the notebook folder within a running notebook
# This cell is not visible when the documentation is built.

from __future__ import annotations

try:
    from _finder import _find_current_folder

    notebook_folder = _find_current_folder()
except ImportError:
    from pathlib import Path

    notebook_folder = Path().cwd()

import logging

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

Hydrological models can differ from one another in terms of required inputs and available functions, but an effort will be made to homogenize them as much as possible as new models get added. Currently, all models have these 3 functions:
- `.run()` which will execute the model, reformat the outputs to be compatible with analysis tools in `xhydro`, then return the simulated streamflows as a `xarray.Dataset`.
  - The streamflow will be called `streamflow` and have units in `m3 s-1`.
  - In the case of 1D data (such as hydrometric stations), that dimension in the dataset will be identified trough a `cf_role: timeseries_id` attribute.
- `.get_inputs()` to retrieve the meteorological inputs.
- `.get_streamflow()` to retrieve the simulated streamflow.

## Initialising the model (e.g. Hydrotel)
The following example will use the Hydrotel model. It is on the more complex side, with most of its parameters hidden within configurations files, but `xhydro` can be used to easily update configuration files, validate the project directory and the meteorological inputs, execute the model, and reformat the outputs to be more inline with CF conventions and other functions within `xhydro`.

Do note that `xhydro` does not prepare the project directory itself, which should be done beforehand. What the class does, when initiating a new instance of `xhydro.modelling.Hydrotel`, is allow control on the entries located in the three main configuration files: the project file, `simulation.csv`, and `output.csv`. The other arguments for the class, as obtained from `get_hydrological_model_inputs`, are listed above. At any time after initialising the class, `update_config()` can be called to update the three configuration files. When called, this function will overwrite the CSV files on disk.

In [ ]:
import os
from pathlib import Path

# The executable depends on the platform
if os.name == "nt":
    executable = "path/to/Hydrotel.exe"
else:
    executable = "hydrotel"

# Prepare the model configuration options
model_config = {
    "model_name": "Hydrotel",
    "project_dir": notebook_folder / "_data" / "example_hydrotel",
    "project_file": "projet.csv",
    "simulation_config": {
        "DATE DEBUT": "2001-01-01",
        "DATE FIN": "2001-12-31",
        "FICHIER STATIONS METEO": "meteo/SLNO_meteo_GC3H.nc",
        "PAS DE TEMPS": 24,
    },
    "output_config": {"TRONCONS": 1, "DEBITS_AVAL": 1},
    "use_defaults": True,
    "executable": executable,
}

In [ ]:
# This is a hidden cell. We'll create a fake Hydrotel directory for the purpose of this example.
import xhydro.testing

xhydro.testing.utils.fake_hydrotel_project(
    notebook_folder / "_data" / "example_hydrotel", meteo=True, debit_aval=True
)

For HYDROTEL, `DATE DEBUT (start date), DATE FIN (end date), and PAS DE TEMPS (timestep frequency)` will always need to be specified, so these need to be added to `simulation_config` if they don't already exist in `simulation.csv`. Additionally, either `FICHIER STATIONS METEO (meteorological stations file)` or `FICHIER GRILLE METEO (meteorological grid file)` need to be specified to guide the model towards the meteorological data.

If using the defaults, streamflow for all river reaches will be outputted. You can modify `output.csv` to change that behaviour.

In [ ]:
# Note that xhm.Hydrotel(**model_config) could also be used to initiate the model.
ht = xhm.hydrological_model(model_config)

print(f"Simulation directory, taken from the project file: '{ht.simulation_dir}'\n")
print(f"Project configuration: '{ht.project_config}'\n")
print(f"Simulation configuration: '{ht.simulation_config}'\n")
print(f"Output configuration: '{ht.output_config}'")

## Validating the meteorological data
A few basic checks will be automatically performed prior to executing hydrological models, but a user might want to perform more advanced health checks (e.g. unrealistic meteorological inputs). This is possible through the use of `xhydro.utils.health_checks`. Consult [the 'xscen' documentation](https://xscen.readthedocs.io/en/latest/notebooks/3_diagnostics.html#Health-checks) for the full list of possible checks.

In this example, we'll make sure that there are no abnormal meteorological values or sequence of values. Since the data used for this example is fake, this will raise some flags.

In [ ]:
health_checks = {
    "raise_on": [],  # If an entry is not here, it will warn the user instead of raising an exception.
    "flags": {
        "pr": {  # You can have specific flags per variable.
            "negative_accumulation_values": {},
            "very_large_precipitation_events": {},
            "outside_n_standard_deviations_of_climatology": {"n": 5},
            "values_repeating_for_n_or_more_days": {"n": 5},
        },
        "tasmax": {
            "tasmax_below_tasmin": {},
            "temperature_extremely_low": {},
            "temperature_extremely_high": {},
            "outside_n_standard_deviations_of_climatology": {"n": 5},
            "values_repeating_for_n_or_more_days": {"n": 5},
        },
        "tasmin": {
            "temperature_extremely_low": {},
            "temperature_extremely_high": {},
            "outside_n_standard_deviations_of_climatology": {"n": 5},
            "values_repeating_for_n_or_more_days": {"n": 5},
        },
    },
}

In [ ]:
# We can use get_inputs() to automatically retrieve the meteorological data. This is very useful for instances like Hydrotel, where this information is hidden within configuration files.
ds_in = ht.get_inputs()
ds_in["pr"].attrs[
    "units"
] = "mm d-1"  # Hydrotel-to-xclim compatibility. Precipitation in xclim needs to be a flux.

# This example will raise warnings, this is normal since we're using fake data.
xh.utils.health_checks(ds_in, **health_checks)

## Executing the model
In most cases, a few basic checkups will be performed prior to executing the model, when the `run()` function is called. In the case of Hydrotel, these checks will be made:

- All files mentioned in the configuration exist.
- The meteorological dataset has the dimensions, coordinates, and variables named in its configuration file (e.g. `SLNO_meteo_GC3H.nc.config`, in this example).
- The dataset has a standard calendar.
- The frequency is uniform (i.e. all time steps are equally spaced).
- The start and end dates are contained in the dataset.
- The dataset is complete (i.e. no missing values).

Only when those checks are satisfied will the function actually execute the model. In addition, specific to Hydrotel, the following arguments can be called:

- `check_missing`: *bool*. Whether to verify for missing data or not. Since this can be very time-consuming, it is False by default.
- `dry_run`: *bool*. Put at True to simply print the command line, without executing it.

Once the model has been executed, `xhydro` will automatically reformat the NetCDF to bring it closer to CF conventions and make it compatible with other `xhydro` modules. Note that for Hydrotel, this reformatting currently only supports the DEBITS_AVAL (outgoing streamflow) output option.

In [ ]:
# For the purpose of this example, we'll leave 'dry_run' as True.
print("Command that would be run in the terminal:")
ht.run(check_missing=True, dry_run=True)

In [ ]:
# This is how the output would look like after reformatting (which was skipped by the dry_run argument)
ht._standardise_outputs()
ht.get_streamflow()

## Model calibration

<div class="alert alert-warning"> <b>WARNING</b>
This is still a work-in-progress. Only the Dummy is currently implemented.
</div>

Model calibration consists of a loop of multiple instances where: model parameters are chosen, the model is run, the results are compared to observations. The calibration functions in `xhydro` rely on `SPOTPY` to perform the optimization process.

When using the calibration module, 2 additional keywords are required for the `model_config`:
- `Qobs`: Contains the observed streamflow used as the calibration target.
- `parameters`: While not necessary to provide this, it is a reserved keyword used by the optimizer.

The calibration function, `xh.modelling.perform_calibration`, has these parameters:

In [ ]:
xh.modelling.perform_calibration?

In [ ]:
import numpy as np

# Prepare the model_config dictionary for the Dummy model
model_config = {
    "model_name": "Dummy",
    "precip": np.array([10, 11, 12, 13, 14, 15]),
    "temperature": np.array([10, 3, -5, 1, 15, 0]),
    "drainage_area": np.array([10]),
    "qobs": np.array([120, 130, 140, 150, 160, 170]),  # Required for calibration
}

# This model has 3 parameters. This will be their possible range.
bounds_low = np.array([0, 0, 0])
bounds_high = np.array([10, 10, 10])

mask = np.array([0, 0, 0, 0, 1, 1])

In [ ]:
# Run the calibration
best_parameters, best_simulation, best_objfun = xhm.perform_calibration(
    model_config,
    obj_func="mae",
    bounds_low=bounds_low,
    bounds_high=bounds_high,
    evaluations=1000,
    algorithm="DDS",
    mask=mask,
    sampler_kwargs={"trials": 1},
)

In [ ]:
# The first output corresponds to the best set of parameters
best_parameters

In [ ]:
# The second output corresponds to the timeseries for the best set of parameters
best_simulation

In [ ]:
# The second output is the value of the objective function for the best set of parameters
best_objfun